In [116]:
using PyPlot
using Optim

In [117]:
# Convert an Float32 rbg image to Float64 grayscale image
function convert_to_grayscale(I::Array{Float32,3})
    I=convert(Array{Float64,3}, I)
    I_gray = 0.2989*I[:,:,1] + 0.5870*I[:,:,2] + 0.1140*I[:,:,3]
    return I_gray::Array{Float64,2}
end

convert_to_grayscale (generic function with 1 method)

In [118]:
# Load Tsukuba disparity dataset and convert it to grayscale
function load_data()
    i0_path = string(@__DIR__,"/i0.png")
    i0 = imread(i0_path)
    i0 = convert_to_grayscale(i0)
    i1_path = string(@__DIR__,"/i1.png")
    i1 = imread(i1_path)
    i1 = convert_to_grayscale(i1)
    gt_path = string(@__DIR__,"/gt.png")
    gt64 = convert(Array{Float64,2}, imread(gt_path)*255)

    @assert maximum(gt64) <= 16
    return i0::Array{Float64,2}, i1::Array{Float64,2}, gt64::Array{Float64,2}
end

load_data (generic function with 1 method)

In [119]:
function log_studentt(x::Float64, alpha::Float64, sigma::Float64)
    
    #TODO: Change the code to accept matrix input
    
    function log_student(x, alpha, sigma)
        return log((1 + (1/2*sigma^2)*x^2)^(-1*alpha))
    end
    
    function log_grad_student(x, alpha, sigma)
        return (-1*alpha*(1/sigma^2)*x) * (1 + (1/2*sigma^2)*x^2)^(-2*alpha - 1)
    end
    
    value = log_student(x, alpha, sigma)
    grad = log_grad_student(x, alpha, sigma)

    return value::Float64, grad::Float64
end

log_studentt (generic function with 1 method)

In [120]:
# Evaluate stereo log prior.
# Set: alpha=1.0, sigma=1.0
function stereo_log_prior(x::Array{Float64,2})
    alpha = 1.0
    sigma = 1.0
    value = 0
    grad = zeros(size(x))
    # Vertical potential
    for i = 1:size(x)[1]-1#Height?
        for j = 1:size(x)[2]#Width?
            value += log_studentt(x[i+1, j] - x[i,j], alpha, sigma)[1]
        end
    end 
    # Horizontal potential
    for i = 1:size(x)[1]#Height?
        for j = 1:size(x)[2]-1#Width?
            value += log_studentt(x[i, j+1] - x[i,j], alpha, sigma)[1]
        end
    end
    # Partial derivative to every pixel
    for k = 1:size(x)[1]
        for l = 1:size(x)[2]
            if k + 1 <= size(x)[1]
                grad[k,l] += log_studentt(x[k,l]-x[k+1,l], alpha, sigma)[2] 
            end
            if k - 1 >= 1
                grad[k,l] += log_studentt(x[k,l]-x[k-1,l], alpha, sigma)[2]
            end
            if l + 1 <= size(x)[2]
                grad[k,l] += log_studentt(x[k,l]-x[k,l+1], alpha, sigma)[2]
            end
            if l - 1 >= 1
                grad[k,l] += log_studentt(x[k,l]-x[k,l-1], alpha, sigma)[2]
            end
        end
    end
    return  value::Float64, grad::Array{Float64,2}
end


stereo_log_prior (generic function with 1 method)

In [121]:
# Calculates the gradient of a given image in both directions
function image_gradient(im::Array{Float64,2})
    horizontal_im_grad = zeros(size(im))
    vertical_im_grad = zeros(size(im))
    for x = 1:size(im)[1]
        for y = 1:size(im)[2]
            if x-1 > 0
                vertical_im_grad[x,y] -= im[x-1,y]
            end
            if x+1 < size(im)[1]
                vertical_im_grad[x,y] += im[x+1,y]
            end
            if y-1 > 0
                horizontal_im_grad[x,y] -= im[x,y-1]
            end
            if y+1 < size(im)[2]
                horizontal_im_grad[x,y] += im[x,y+1]
            end
        end
    end
    return horizontal_im_grad::Array{Float64,2}, vertical_im_grad::Array{Float64,2}
end

image_gradient (generic function with 1 method)

In [122]:
# Evaluate stereo log likelihood.
# Set: Alpha = 1.0, Sigma = 0.004
function stereo_log_likelihood(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})
    alpha = 1.0
    sigma = 0.004
    value = 0
    grad = zeros(size(im1))
    # We need the horizontal image derivative from I1 to calculate the gradient of the LH
    h_img_grad = image_gradient(im1)[1]
    for i = 1:size(x)[1]
        for j = 1:size(x)[2]
            d = im0[i,j]-im1[i-trunc(Int, x[i,j]), j]
            value += log_studentt(d, alpha, sigma)[1]
            grad[i,j] = (-1)*log_studentt(d, alpha, sigma)[2]*h_img_grad[i-trunc(Int, x[i,j]), j]          
        end
    end
    return value::Float64, grad::Array{Float64,2}
end

stereo_log_likelihood (generic function with 1 method)

In [123]:
# Evaluate stereo posterior
function stereo_log_posterior(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2}) 
    #log(posterior) = log(prior*LH) = log(prior)*log(LH)
    # (We can drop the marginalisation terms)
    log_posterior = stereo_log_likelihood(x, im0, im1)[1] * stereo_log_prior(x)[1] * stereo_log_prior(im0)[1]
    grad_lh = stereo_log_likelihood(x, im0, im1)[2]
    grad_x = stereo_log_prior(x)[2]
    # (Derivative of I0 to x should be zero, so we drop it ...)
    log_posterior_grad = grad_lh + grad_x
    return log_posterior::Float64, log_posterior_grad::Array{Float64,2}
end

stereo_log_posterior (generic function with 1 method)

In [124]:
# Run stereo algorithm using gradient ascent or sth similar
function stereo(x0::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})

    # Estimate disparity map
    x = optimize(stereo_log_posterior, (x0, im0, im1), BFGS())

    return x::Array{Float64,2}
end

stereo (generic function with 1 method)

In [125]:
function problem3()
    # use problem 2's load_data
    im0, im1, gt = load_data()
    
    print("Im0: ", size(im0))
    print("\nIm1: ", size(im1))
    print("\nGT: ", size(gt))
    print("\nGrad of log P(im1): ", size(stereo_log_prior(im1)[2]))
    print("\nGrad of log P(im1|im0,gt): ", size(stereo_log_likelihood(gt, im0, im1)[2]))
    print("\nGrad of log P(im0,gt|im1): ", size(stereo_log_posterior(gt, im0, im1)[2]))
    # Display stereo: Initialized with constant 8's


    # Display stereo: Initialized with noise in [0,14]


    # Display stereo: Initialized with gt
    estimated_disparity_gt = stereo(gt, im0, im1)
    imshow(estimated_disparity)

    # Coarse to fine estimation..


end


problem3 (generic function with 1 method)

In [126]:
problem3()

Im0: (288, 384)
Im1: (288, 384)
GT: (288, 384)
Grad of log P(im1): (288, 384)
Grad of log P(im1|im0,gt): (288, 384)
Grad of log P(im0,gt|im1): (288, 384)

MethodError: MethodError: no method matching optimize(::typeof(stereo_log_posterior), ::Tuple{Array{Float64,2},Array{Float64,2},Array{Float64,2}}, ::BFGS{LineSearches.InitialStatic{Float64},LineSearches.HagerZhang{Float64,Base.RefValue{Bool}},getfield(Optim, Symbol("##17#19"))})
Closest candidates are:
  optimize(::Any, ::Any, ::Any, !Matched::AbstractArray; inplace, autodiff, kwargs...) at /home/yannik/.julia/packages/Optim/Agd3B/src/multivariate/optimize/interface.jl:78
  optimize(::Any, ::Any, ::Any, !Matched::AbstractArray, !Matched::Optim.Options; inplace, autodiff) at /home/yannik/.julia/packages/Optim/Agd3B/src/multivariate/optimize/interface.jl:105
  optimize(::Any, ::Any, ::Any, !Matched::AbstractArray{T,N} where N, !Matched::Optim.AbstractOptimizer) where T at /home/yannik/.julia/packages/Optim/Agd3B/src/multivariate/optimize/interface.jl:128
  ...